# Toxic Comment Classification Challenge

In [1]:
# Installing the modules

!pip3 install kaggle
!pip install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip

     |████████████████████████████████| 25.9 MB 120 kB/s 
     |████████████████████████████████| 10.1 MB 5.2 MB/s 
     |████████████████████████████████| 596 kB 51.4 MB/s 
     |████████████████████████████████| 102 kB 11.8 MB/s 
     |████████████████████████████████| 675 kB 66.3 MB/s 
     |████████████████████████████████| 4.7 MB 23.0 MB/s 
     |████████████████████████████████| 62 kB 609 kB/s 
     |████████████████████████████████| 812 kB 61.4 MB/s 
     |████████████████████████████████| 38.2 MB 22 kB/s 
  Created wheel for pandas-profiling: filename=pandas_profiling-3.1.1-py2.py3-none-any.whl size=261287 sha256=dfbd6a3833100c2008b6c9c241955854fa85d670996bf5a1b884991795688c11
  Stored in directory: /tmp/pip-ephem-wheel-cache-26haiuy6/wheels/cc/d5/09/083fb07c9363a2f45854b0e3a7de7d7c560f07da74b9e9769d
  Created wheel for htmlmin: filename=htmlmin-0.1.12-py3-none-any.whl size=27098 sha256=dece16c936e8974b6314b3f4b5a813454450f9cbbfebe6cc11a6d37c0d755dfd
  Stored in directory: /roo

In [32]:
# Required modules

import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow_hub as hub

from lightgbm import LGBMClassifier

from sklearn.multioutput import MultiOutputClassifier

from zipfile import ZipFile
from google.colab import files
from matplotlib import pyplot as plt

In [3]:
# Config

%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 7)

In [4]:
# Upload the files

files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"anurag1817","key":"09b66f16e6edda0614443b2a02389465"}'}

In [5]:
# Save the files

!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [6]:
# Download the dataset

!kaggle competitions download -c jigsaw-toxic-comment-classification-challenge

 49% 13.0M/26.3M [00:00<00:00, 132MB/s]
100% 26.3M/26.3M [00:00<00:00, 128MB/s]
  0% 0.00/1.46M [00:00<?, ?B/s]
100% 1.46M/1.46M [00:00<00:00, 98.6MB/s]
  0% 0.00/1.39M [00:00<?, ?B/s]
100% 1.39M/1.39M [00:00<00:00, 91.8MB/s]
 77% 18.0M/23.4M [00:00<00:00, 45.1MB/s]
100% 23.4M/23.4M [00:00<00:00, 52.8MB/s]


In [7]:
# Extracting individual files

with ZipFile('/content/train.csv.zip', 'r') as zf:
    zf.extractall('./')

with ZipFile('/content/test.csv.zip', 'r') as zf:
    zf.extractall('./')

with ZipFile('/content/test_labels.csv.zip', 'r') as zf:
    zf.extractall('./')

with ZipFile('/content/sample_submission.csv.zip', 'r') as zf:
    zf.extractall('./')

In [8]:
# Loading the train data

train = pd.read_csv('train.csv')
train.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [16]:
# Inspecting the data

train.info()
train.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 8 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   id             159571 non-null  object
 1   comment_text   159571 non-null  object
 2   toxic          159571 non-null  int64 
 3   severe_toxic   159571 non-null  int64 
 4   obscene        159571 non-null  int64 
 5   threat         159571 non-null  int64 
 6   insult         159571 non-null  int64 
 7   identity_hate  159571 non-null  int64 
dtypes: int64(6), object(2)
memory usage: 9.7+ MB


,toxic,severe_toxic,obscene,threat,insult,identity_hate
count,159571.000000,159571.000000,159571.000000,159571.000000,159571.000000,159571.000000
mean,0.095844,0.009996,0.052948,0.002996,0.049364,0.008805
std,0.294379,0.099477,0.223931,0.054650,0.216627,0.093420
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [17]:
# Load the test data

test = pd.read_csv('test.csv')
test.head()

,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap..."
3,00017563c3f7919a,":If you have a look back at the source, the in..."
4,00017695ad8997eb,I don't anonymously edit articles at all.


In [9]:
# Downloading the embedder

embed = hub.load('https://tfhub.dev/google/nnlm-en-dim50/2')

In [18]:
# Get the embeddings

train_embed = embed(train['comment_text'].values)
test_embed = embed(test['comment_text'].values)

In [33]:
# Fitting a model

model = LGBMClassifier()
final_model = MultiOutputClassifier(model)
final_model.fit(train_embed, train[['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']])

MultiOutputClassifier(estimator=LGBMClassifier())

In [34]:
# Prediction on test set

test_preds = final_model.predict(test_embed)

In [35]:
# Making a final dataframe

submission = pd.DataFrame(columns=['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'], data=test_preds)
submission['id'] = test['id']
submission.to_csv('output.csv', index=False)

In [36]:
# Submission via API

!kaggle competitions submit -c jigsaw-toxic-comment-classification-challenge -f output.csv -m "LGBM Output"

100% 4.24M/4.24M [00:00<00:00, 14.0MB/s]
Successfully submitted to Toxic Comment Classification Challenge